In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Create an instance of TavilySearchResults with k=6 for retrieving up to 6 search results
search_tool = TavilySearchResults(
    max_results=10, search_depth="basic"
)  # basic or advanced

In [3]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field


class Info(BaseModel):
    name: str = Field(..., description="식당 이름")
    address: str = Field(..., description="식당 주소")
    subway: str = Field(..., description="식당 지하철역")
    lat: str = Field(..., description="식당 위도")
    lng: str = Field(..., description="식당 경도")
    menu: str = Field(..., description="식당 메뉴")


class Answer(BaseModel):
    answer: str = Field(..., description="답변 내용")
    infos: list[Info] = Field(..., description="맛집 정보 리스트")


parser = JsonOutputParser(pydantic_object=Answer)

In [4]:
# 템플릿 수정: search_results를 제거하거나 처리 방법 변경
TEMPLATE = """당신은 맛집을 찾아서 정리하는 블로거 입니다.

사용자 질문에 따른 맛집 정보를 search_tool을 사용하여 검색 후 정리하여 데이터를 구축합니다.

그 후 확인된 주소를 기반으로 get_coordinates 도구를 사용하여 좌표를 추출해주세요.

추출된 좌표를 기반으로 get_subway_info 도구를 사용하여 지하철역 정보를 추출해주세요.

사용자 질문:
{input}

출력 형식:

{{
    "answer": "아주 간단한 답변 내용",
    "infos": [
        {{
            "name": "식당 이름",
            "address": "식당 주소",
            "subway": "식당 지하철역",
            "lat": "식당 위도",
            "lng": "식당 경도",
            "menu": "메뉴1, 메뉴2, ...",
            "review": "식당 후기",
            "url": "참고한 url"
        }},
        {{
            "name": "식당 이름2",
            "address": "식당 주소2",
            "subway": "식당 지하철역2",
            "lat": "식당 위도2",
            "lng": "식당 경도2",
            "menu": "메뉴1, 메뉴2, ...",
            "review": "식당 후기2",
            "url": "참고한 url2"
        }},
        ...
    ]
}}
{agent_scratchpad}
"""

from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests
import os

prompt = ChatPromptTemplate.from_template(template=TEMPLATE)

# LLM 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)


# search_tool을 LangChain 형식에 맞게 변환
@tool
def search(query: str) -> list:
    """Search for information about restaurants and food."""
    return search_tool.invoke(query)


@tool
def get_coordinates(address: str) -> list:
    """Get the coordinates of a restaurant."""
    coordinate_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {os.getenv('KAKAO_API_KEY')}"}
    params = {"query": address}
    response = requests.get(coordinate_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
    if data["documents"]:
        location = data["documents"][0]
        latitude = location["y"]
        longitude = location["x"]

        return latitude, longitude


@tool
def get_subway_info(latitude: str, longitude: str) -> list:
    """Get the subway information of a restaurant."""
    station_url = "https://dapi.kakao.com/v2/local/search/category.json"
    headers = {"Authorization": f"KakaoAK {os.getenv('KAKAO_API_KEY')}"}
    params = {
        "category_group_code": "SW8",
        "x": longitude,
        "y": latitude,
        "radius": 2000,
        "sort": "distance",
    }
    response = requests.get(station_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data["documents"]:
            station_name = data["documents"][0]["place_name"]
            station_distance = data["documents"][0]["distance"]
            return station_name, station_distance
        else:
            return "정보 없음", "정보 없음"

In [5]:
# 도구 리스트 생성
tools = [search, get_coordinates, get_subway_info]

# 에이전트 생성
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 에이전트 실행 예제
result = agent_executor.invoke({"input": "포천 맛집에 대해서 많이 찾아줘"})



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': '포천 맛집'}`


[{'url': 'https://tongtonginformation.com/entry/포천-맛집-베스트10-현지인-추천', 'content': "포천 맛집 베스트10 5. 참나무쟁이: 훈제의 깊은 맛 '참나무쟁이'는 훈제 요리를 전문으로 하는 포천 맛집으로, 참나무로 직접 훈제한 고기의 깊은 맛을 자랑합니다.이곳의 대표 메뉴인 '훈제오리'는 바삭하면서도 촉촉한 식감이 일품이며, 특별한 소스와 함께 제공되어 더욱 맛있습니다."}, {'url': 'https://tour.click1-tip.com/entry/포천-맛집-베스트10', 'content': '오늘은 산정호수와 국립수목원으로 유명한 경기도 포천시의 소문난 음식점들을 모아서 포천 맛집 베스트 10으로 소개해드리겠습니다. 대표적인 포천 먹거리인 이동갈비를 포함해 가성비 좋은 포천 현지인 맛집과 로컬 식당 중심으로 객관적인 포천 맛집을 정리해 드리겠습니다. 기본적인 맛집 선정'}, {'url': 'https://jdblue2022.tistory.com/entry/포천-맛집-베스트10', 'content': '오늘은 산정호수와 명성산으로 유명한 경기도 포천의 소문난 맛집 베스트 10곳을 소개해드리고자 하는데 양대 포털의 플레이스를 중심으로 현지에서 유명한 포천 먹거리인 이동갈비와 두부에 포천 현지인 맛집들도 같이 묶어서 포천 맛집을 정리해드리고자 합니다. 기본적인 선정 기준은 양대 검색'}, {'url': 'https://lulutourstory.tistory.com/entry/경기도-포천-가볼만한곳-맛집-BEST-8-추천', 'content': '1. 삼낙촌주소경기 포천시 소흘읍 죽엽산로 559영업시간10:30 - 20:00, 15:50 - 16:30 브레이크타임, 19:00 라스트오더 삼낙촌은 경기도 포천시 소흘읍에 위치한 활낙지 전문점입니다. 신선한 

In [6]:
print(result["output"])

```json
{
    "answer": "포천의 맛집 정보를 정리했습니다.",
    "infos": [
        {
            "name": "삼낙촌",
            "address": "경기 포천시 소흘읍 죽엽산로 559",
            "subway": "정보 없음",
            "lat": "37.7848571681541",
            "lng": "127.164328773062",
            "menu": "활낙지",
            "review": "신선한 해산물과 정성스럽게 준비한 반찬들로 유명한 삼낙촌은 포천을 방문하는 관광객들과 현지인들에게 모두 사랑받는 곳입니다.",
            "url": "https://lulutourstory.tistory.com/entry/경기도-포천-가볼만한곳-맛집-BEST-8-추천"
        },
        {
            "name": "이동갈비",
            "address": "포천시 이동면",
            "subway": "정보 없음",
            "lat": "38.0306695650759",
            "lng": "127.367579322942",
            "menu": "이동갈비",
            "review": "포천의 대표적인 먹거리로, 가성비 좋은 포천 현지인 맛집으로 추천됩니다.",
            "url": "https://tour.click1-tip.com/entry/포천-맛집-베스트10"
        }
    ]
}
```
